In [1]:
import sys
sys.path.append('../')
import pandas as pd
from roverdata.db import DataWarehouse
from main import main
from taxes_increase import utils

## Get Data

In [ ]:
#LTV, retrans = main()

2019-09-17 15:00:37,388 main         INFO     fetching LTV data


In [ ]:
LTV = pd.from_csv('LTV.csv', sep = "\t")
retrans.to_csv('retrans.csv')

In [ ]:
LTV.head()

## Calculate Percent GMV Drop for Breakeven

In [ ]:
fee=pd.Series([.035,.07],name="Fee Increase")
tax = pd.Series('No tax/4% tax/8% tax'.split('/'),name='Sales Tax')
#pd.DataFrame(columns=['Relative Boost to Rover take','Breakeven loss to GMV'])
x=pd.DataFrame(fee).assign(tag=1)
x=pd.DataFrame(tax).assign(tag=1).merge(x).drop(columns='tag').assign(tax=[0,0,4,4,8,8])
x['Relative Boost to Rover take']=x['Fee Increase']/(27-x.tax)*100
x['Breakeven loss to GMV']=(1+x['Relative Boost to Rover take'])**-1-1
x['Relative Boost to Rover take']=x['Relative Boost to Rover take'].apply('{:.1%}'.format)
x['Breakeven loss to GMV']=x['Breakeven loss to GMV'].apply('{:.1%}'.format)
x=x.drop(columns='tax').pivot(index='Fee Increase',columns='Sales Tax')
x

## Mean and Bootstrapped Standard Error

In [ ]:
LTV180 = LTV[LTV.dates==180].drop(columns=['dates','requester_id'])
LTV180_new = LTV180[LTV180.new_repeat=='new']
LTV180_new_account = LTV180[LTV180.new_account=='new']
splitter = ['service']

In [ ]:
np.random.seed(9)
boots_new = utils.bootstrap(LTV180_new,samplesize=10000,nsamples=1000).groupby('service sample_id'.split()).mean()
means_new = boots_new.groupby(splitter).mean().round(2)
means_new

In [ ]:
np.random.seed(9)
boots_new_account = utils.bootstrap(LTV180_new_account,samplesize=10000,nsamples=1000).groupby('service sample_id'.split()).mean()
means_new_account = boots_new.groupby(splitter).mean().round(2)
means_new_account

In [ ]:
var10k = boots.groupby(splitter).var()
propSE10k = var10k**.5/means
propSE10k

## Get Customer Volume

In [ ]:
retrans.head()